# OUTDATED

This is an example from sync version.

Awaiting subsampling from reqstore

Load libraries

In [1]:
import os
import hydro_serving_grpc as hs

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn
import requests
import time
from tqdm import tqdm_notebook as tqdm

In [2]:
from sklearn.preprocessing import LabelEncoder
import sklearn.model_selection

Get adult data

In [13]:
# Helepr function to process the data
def map_array_values(series, value_map):
    if series.dtype == 'object':
        ret = series.str.strip().copy()
    else:
        ret = series.copy()
    for src, target in value_map.items():
        ret[ret == src] = target
    return ret


# Readble feature names
feature_names = ["Age", "Workclass", "fnlwgt", "Education",
                 "Education-Num", "Marital Status", "Occupation",
                 "Relationship", "Race", "Sex", "Capital Gain",
                 "Capital Loss", "Hours per week", "Country", 'Income']
features_to_use = [0, 1, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13]
categorical_features = [1, 3, 5, 6, 7, 8, 9, 10, 11, 13]
education_map = {
    '10th': 'Dropout', '11th': 'Dropout', '12th': 'Dropout', '1st-4th':
        'Dropout', '5th-6th': 'Dropout', '7th-8th': 'Dropout', '9th':
        'Dropout', 'Preschool': 'Dropout', 'HS-grad': 'High School grad',
    'Some-college': 'High School grad', 'Masters': 'Masters',
    'Prof-school': 'Prof-School', 'Assoc-acdm': 'Associates',
    'Assoc-voc': 'Associates',
}
occupation_map = {
    "Adm-clerical": "Admin", "Armed-Forces": "Military",
    "Craft-repair": "Blue-Collar", "Exec-managerial": "White-Collar",
    "Farming-fishing": "Blue-Collar", "Handlers-cleaners":
        "Blue-Collar", "Machine-op-inspct": "Blue-Collar", "Other-service":
        "Service", "Priv-house-serv": "Service", "Prof-specialty":
        "Professional", "Protective-serv": "Other", "Sales":
        "Sales", "Tech-support": "Other", "Transport-moving":
        "Blue-Collar",
}
country_map = {
    'Cambodia': 'SE-Asia', 'Canada': 'British-Commonwealth', 'China':
        'China', 'Columbia': 'South-America', 'Cuba': 'Other',
    'Dominican-Republic': 'Latin-America', 'Ecuador': 'South-America',
    'El-Salvador': 'South-America', 'England': 'British-Commonwealth',
    'France': 'Euro_1', 'Germany': 'Euro_1', 'Greece': 'Euro_2',
    'Guatemala': 'Latin-America', 'Haiti': 'Latin-America',
    'Holand-Netherlands': 'Euro_1', 'Honduras': 'Latin-America',
    'Hong': 'China', 'Hungary': 'Euro_2', 'India':
        'British-Commonwealth', 'Iran': 'Other', 'Ireland':
        'British-Commonwealth', 'Italy': 'Euro_1', 'Jamaica':
        'Latin-America', 'Japan': 'Other', 'Laos': 'SE-Asia', 'Mexico':
        'Latin-America', 'Nicaragua': 'Latin-America',
    'Outlying-US(Guam-USVI-etc)': 'Latin-America', 'Peru':
        'South-America', 'Philippines': 'SE-Asia', 'Poland': 'Euro_2',
    'Portugal': 'Euro_2', 'Puerto-Rico': 'Latin-America', 'Scotland':
        'British-Commonwealth', 'South': 'Euro_2', 'Taiwan': 'China',
    'Thailand': 'SE-Asia', 'Trinadad&Tobago': 'Latin-America',
    'United-States': 'United-States', 'Vietnam': 'SE-Asia'
}
married_map = {
    'Never-married': 'Never-Married', 'Married-AF-spouse': 'Married',
    'Married-civ-spouse': 'Married', 'Married-spouse-absent':
        'Separated', 'Separated': 'Separated', 'Divorced':
        'Separated', 'Widowed': 'Widowed'
}


# Transform Continiouse Cap Gains into the discrete variable
def cap_gains_fn(x):
    x = x.astype(float)
    d = np.digitize(x, [0, np.median(x[x > 0]), float('inf')], right=True)
    new_series = pd.Series(["None"] * len(d))
    new_series[d == 0] = 'None'
    new_series[d == 1] = 'Low'
    new_series[d == 2] = 'High'
    return new_series

# Specify transofrmations for each column
transformations = {
    'Education': lambda x: map_array_values(x, education_map),
    'Marital Status': lambda x: map_array_values(x, married_map),
    'Occupation': lambda x: map_array_values(x, occupation_map),
    'Capital Gain': cap_gains_fn,
    'Capital Loss': cap_gains_fn,
    'Country': lambda x: map_array_values(x, country_map),
}

# Load df
df = pd.read_csv("../anchor2/anchor2/examples/data/adult/adult.data", header=None)
df.columns = feature_names
target_labels = pd.Series(df.iloc[:, -1], index=df.index)
df = df.iloc[:, features_to_use]
df.dropna(inplace=True)

# Transform features
for feature, fun in transformations.items():
    df[feature] = fun(df[feature])

# Store dictionary with {Category id -> category classes}
categorical_features_idx = [1, 2, 3, 4, 5, 6, 7, 8, 9, 11]
categorical_names = {}  # Dictionary with (Category id -> category classes)
for f_idx in categorical_features_idx:
    le = LabelEncoder()
    df.iloc[:, f_idx] = le.fit_transform(df.iloc[:, f_idx])
    categorical_names[f_idx] = le.classes_

# Encode target label
le = LabelEncoder()
target_labels = le.fit_transform(target_labels)
class_names = list(le.classes_)

# Split the dataset into train\val\test
train_X, rest_X, train_y, rest_y = sklearn.model_selection.train_test_split(df, target_labels, stratify=target_labels,
                                                                            test_size=0.5, random_state=42)
val_X, test_X, val_y, test_y = sklearn.model_selection.train_test_split(rest_X, rest_y, stratify=rest_y,
                                                                        test_size=0.5, random_state=42)

Check random sample

In [14]:
idx = np.random.choice(range(test_X.shape[0]))
x = test_X.iloc[idx]
print(x)

Age               45
Workclass          4
Education          4
Marital Status     2
Occupation         6
Relationship       3
Race               4
Sex                1
Capital Gain       2
Capital Loss       2
Hours per week    45
Country            9
Name: 17180, dtype: int64


## Connect to HS

Specify the hydrosphere inference endpoint

In [15]:
service_link = "https://dev.k8s.hydrosphere.io/gateway/application/adult"

In [16]:
# Function to store sample in a json with signature 
def make_signatured_json(sample):
    output_json = {}
    if type(sample) == pd.Series:
        feature_names = sample.index
        values = sample
    else:
        feature_names = [
            "Age",
            "Workclass",
            "Education",
            "Marital Status",
            "Occupation",
            "Relationship",
            "Race",
            "Sex",
            "Capital Gain",
            "Capital Loss",
            "Hours per week",
            "Country"]
        values = sample
        
    for feature_idx, fname in enumerate(feature_names):
        output_json[fname] = [int(v) for v in values.loc[:, fname]]
    return output_json

def make_proto(sample):
    j = make_signatured_json(sample)
    keys = j.keys()
    tensor_shapes = [hs.TensorShapeProto(dim=[hs.TensorShapeProto.Dim(size=len(j[k])), hs.TensorShapeProto.Dim(size=1)])  for k in keys ]
    tensors = [hs.TensorProto(dtype=hs.DT_INT64, int64_val = j[k], tensor_shape = ts) for k, ts in zip(keys, tensor_shapes) ]
    return dict(zip(keys, tensors))


In [17]:
inp = make_proto(test_X.iloc[idx:idx+5])
inp

{'Age': dtype: DT_INT64
 tensor_shape {
   dim {
     size: 5
   }
   dim {
     size: 1
   }
 }
 int64_val: 45
 int64_val: 53
 int64_val: 30
 int64_val: 63
 int64_val: 25, 'Workclass': dtype: DT_INT64
 tensor_shape {
   dim {
     size: 5
   }
   dim {
     size: 1
   }
 }
 int64_val: 4
 int64_val: 4
 int64_val: 4
 int64_val: 6
 int64_val: 4, 'Education': dtype: DT_INT64
 tensor_shape {
   dim {
     size: 5
   }
   dim {
     size: 1
   }
 }
 int64_val: 4
 int64_val: 4
 int64_val: 4
 int64_val: 6
 int64_val: 1, 'Marital Status': dtype: DT_INT64
 tensor_shape {
   dim {
     size: 5
   }
   dim {
     size: 1
   }
 }
 int64_val: 2
 int64_val: 0
 int64_val: 2
 int64_val: 0
 int64_val: 2, 'Occupation': dtype: DT_INT64
 tensor_shape {
   dim {
     size: 5
   }
   dim {
     size: 1
   }
 }
 int64_val: 6
 int64_val: 8
 int64_val: 8
 int64_val: 5
 int64_val: 6, 'Relationship': dtype: DT_INT64
 tensor_shape {
   dim {
     size: 5
   }
   dim {
     size: 1
   }
 }
 int64_val: 3
 int64_val

Get response for an example, HTTP

In [18]:
response = requests.post(url=service_link, json=make_signatured_json(test_X.iloc[idx:idx+50]))
prediction = np.array(response.json()["Prediction"])
print(f"Predicted label is {class_names[prediction[0][0]]}")

Predicted label is  <=50K


Check through plain GPRC

In [20]:
import grpc
import hydro_serving_grpc as hs

creds = grpc.ssl_channel_credentials()
channel = grpc.secure_channel('dev.k8s.hydrosphere.io', creds)
client = hs.PredictionServiceStub(channel)

In [21]:
model_spec = hs.ModelSpec(name='adult', signature_name = 'adult')

In [22]:
inp = make_proto(test_X.iloc[idx:idx+5])

In [24]:
request = hs.PredictRequest(model_spec=model_spec, inputs=inp)
result = client.Predict(request)
print(result)

outputs {
  key: "Prediction"
  value {
    dtype: DT_INT64
    tensor_shape {
      dim {
        size: -1
      }
      dim {
        size: 1
      }
    }
    int64_val: 0
    int64_val: 1
    int64_val: 0
    int64_val: 1
    int64_val: 0
  }
}



Check through GPRC without Sonar attached to it

In [86]:
import grpc
import hydro_serving_grpc as hs
import hydro_serving_grpc.gateway as hsg

# creds = grpc.ssl_channel_credentials()
channel = grpc.insecure_channel('localhost')
client = hsg.GatewayServiceStub(channel)

model_spec = hs.ModelSpec(name='adult-salary-prediction', signature_name ="predict")
model_spec.version.value = 1
inp = make_proto(test_X.iloc[idx:idx+5])
request = hs.PredictRequest(model_spec=model_spec, inputs=inp)

client.PredictModelOnly(request)

_Rendezvous: <_Rendezvous of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "failed to connect to all addresses"
	debug_error_string = "{"created":"@1560518649.152660000","description":"Failed to pick subchannel","file":"src/core/ext/filters/client_channel/client_channel.cc","file_line":3381,"referenced_errors":[{"created":"@1560518649.152521000","description":"failed to connect to all addresses","file":"src/core/ext/filters/client_channel/lb_policy/pick_first/pick_first.cc","file_line":453,"grpc_status":14}]}"
>

fetch model contract


In [81]:
def get_model_id(application_name: str):
    SERVING_URL = "http://localhost"
    r = requests.get(f"{SERVING_URL}/api/v2/application/{application_name}")
#     print(f"{SERVING_URL}/api/v2/application/{application_name}")
#     print(r.json())
    model = r.json()['executionGraph']['stages'][0]['modelVariants'][0]['modelVersion']['model']
    return model

In [83]:
m = get_model_id("adult")
m

{'id': 1, 'name': 'adult-salary-prediction'}

In [95]:
from hydro_serving_grpc import manager as hsm

# creds = grpc.ssl_channel_credentials()
channel = grpc.insecure_channel('127.0.0.1:9090')
request = hsm.GetVersionRequest(id=m['id'])
client = hsm.ManagerServiceStub(channel)
client.GetVersion(request)

id: 1
version: 1
status: "Released"
model {
  id: 1
  name: "adult-salary-prediction"
}
contract {
  model_name: "model"
  predict {
    signature_name: "predict"
    inputs {
      name: "Age"
      shape {
        dim {
          size: -1
        }
        dim {
          size: 1
        }
      }
      dtype: DT_INT64
    }
    inputs {
      name: "Workclass"
      shape {
        dim {
          size: -1
        }
        dim {
          size: 1
        }
      }
      dtype: DT_INT64
    }
    inputs {
      name: "Education"
      shape {
        dim {
          size: -1
        }
        dim {
          size: 1
        }
      }
      dtype: DT_INT64
    }
    inputs {
      name: "Marital Status"
      shape {
        dim {
          size: -1
        }
        dim {
          size: 1
        }
      }
      dtype: DT_INT64
    }
    inputs {
      name: "Occupation"
      shape {
        dim {
          size: -1
        }
        dim {
          size: 1
        }
      }
     

In [106]:
import hydro_serving_grpc

In [113]:
list(filter(lambda x: x.startswith("DT_"), dir(hydro_serving_grpc)))

['DT_BFLOAT16',
 'DT_BOOL',
 'DT_COMPLEX128',
 'DT_COMPLEX64',
 'DT_DOUBLE',
 'DT_FLOAT',
 'DT_HALF',
 'DT_INT16',
 'DT_INT32',
 'DT_INT64',
 'DT_INT8',
 'DT_INVALID',
 'DT_MAP',
 'DT_QINT16',
 'DT_QINT32',
 'DT_QINT8',
 'DT_QUINT16',
 'DT_QUINT8',
 'DT_RESOURCE',
 'DT_STRING',
 'DT_UINT16',
 'DT_UINT32',
 'DT_UINT64',
 'DT_UINT8',
 'DT_VARIANT']

In [116]:
for k, vs in np.sctypes.items():
    for v in vs:
        print(str(v))

<class 'numpy.int8'>
<class 'numpy.int16'>
<class 'numpy.int32'>
<class 'numpy.int64'>
<class 'numpy.uint8'>
<class 'numpy.uint16'>
<class 'numpy.uint32'>
<class 'numpy.uint64'>
<class 'numpy.float16'>
<class 'numpy.float32'>
<class 'numpy.float64'>
<class 'numpy.float128'>
<class 'numpy.complex64'>
<class 'numpy.complex128'>
<class 'numpy.complex256'>
<class 'bool'>
<class 'object'>
<class 'bytes'>
<class 'str'>
<class 'numpy.void'>


In [96]:
from google.protobuf.json_format import MessageToDict
modelVersion = MessageToDict(client.GetVersion(request))

In [102]:
contract = modelVersion['contract']
signature_name = contract['predict']['signatureName']
inputs = contract['predict']['inputs']
outputs = contract['predict']['outputs']

In [103]:
signature_name

'predict'

In [69]:
type(pd.DataFrame(np.array([[1,2 ], [3,4 ]])).dtypes[0])

numpy.dtype

In [98]:
batch_dim_present = False

shapes = [tuple([int(d['size']) for d in i['shape']['dim']]) for i in inputs]
dtypes = [i['dtype'] for i in inputs]
feature_names = [i['name'] for i in inputs]
# TODO assert dtypes!

# Check for presences batch_dim
if any([s[0]== -1 for s in shapes]):
        assert all([s[0]== -1 for s in shapes]), "All columns must contain batch-dim"
        batch_dim_present = True

# Check that data is either 2D (including batch dim) or 1D (without batch dim)
if any([len(s) > 2 for s in shapes]) or any([len(s) > 1 for s in shapes]) and not batch_dim_present:
    raise ValueError("Store tensor data in 2D with batch_dim=-1 or either 1D")
    
# Check that data is either series of scalar, [[-1, 1], [-1, 1], ...], [-1, X] or just [X]
is_column_defined = all([s == (-1, 1) for s in shapes]) or all(s==('scalar',) for s in shapes)
is_tensor_defined = len(shapes) == 1

if not (is_column_defined or is_tensor_defined):
    raise ValueError("Can not comprehend contract")

if is_tensor_defined:
    shape = shapes[0]
    name = inputs[0]['name']
    feature_names = [f"name_{i}" for i in range(shape[1])]
    dtype = dtypes[0]
    del dtypes, shapes


In [65]:
"prediction" in [o['name'] for o in outputs]

False

In [111]:
shapes = [tuple([int(d['size']) for d in i['shape']['dim']]) for i in outputs]
shapes

[(-1, 1)]

# Anchor

Specify the link to Anchor explanation service

In [8]:
anchor_link = "http://0.0.0.0:5000/anchor" 

Prepare configs

In [9]:
adult_anchor_config = {
  "precision_threshold": 0.95,
  "verbose": False,
  "ordinal_features_idx": [
    0,
    10
  ],
  "oh_encoded_categories": {},
  "label_decoders": {
    "1": [
      " ?",
      " Federal-gov",
      " Local-gov",
      " Never-worked",
      " Private",
      " Self-emp-inc",
      " Self-emp-not-inc",
      " State-gov",
      " Without-pay"
    ],
    "2": [
      "Associates",
      "Bachelors",
      "Doctorate",
      "Dropout",
      "High School grad",
      "Masters",
      "Prof-School"
    ],
    "3": [
      "Married",
      "Never-Married",
      "Separated",
      "Widowed"
    ],
    "4": [
      "?",
      "Admin",
      "Blue-Collar",
      "Military",
      "Other",
      "Professional",
      "Sales",
      "Service",
      "White-Collar"
    ],
    "5": [
      " Husband",
      " Not-in-family",
      " Other-relative",
      " Own-child",
      " Unmarried",
      " Wife"
    ],
    "6": [
      " Amer-Indian-Eskimo",
      " Asian-Pac-Islander",
      " Black",
      " Other",
      " White"
    ],
    "7": [
      " Female",
      " Male"
    ],
    "8": [
      "High",
      "Low",
      "None"
    ],
    "9": [
      "High",
      "Low",
      "None"
    ],
    "11": [
      "?",
      "British-Commonwealth",
      "China",
      "Euro_1",
      "Euro_2",
      "Latin-America",
      "Other",
      "SE-Asia",
      "South-America",
      "United-States",
      "Yugoslavia"
    ]
  },
  "strategy": "kl-lucb",
  "feature_names": [
    "Age",
    "Workclass",
    "Education",
    "Marital Status",
    "Occupation",
    "Relationship",
    "Race",
    "Sex",
    "Capital Gain",
    "Capital Loss",
    "Hours per week",
    "Country"
  ]
}


Populate reqstore with data

In [10]:
service_link = "https://dev.k8s.hydrosphere.io/gateway/application/adult-salary-app"
for idx in tqdm(range(test_X.shape[0]//500 - 1)):
    sample = test_X.iloc[idx*500 :(idx +1)*500]    
    response = requests.post(url=service_link, json=make_signatured_json(sample))
    print(response.status_code, end=".. ")
#     time.sleep(0.05)

200.. 200.. 200.. 200.. 200.. 200.. 200.. 200.. 200.. 200.. 200.. 200.. 200.. 200.. 200.. 


## Example 0

In [11]:
idx = np.random.choice(range(test_X.shape[0]))
x = test_X.iloc[idx]
print(x)

Age               53
Workclass          2
Education          3
Marital Status     0
Occupation         2
Relationship       0
Race               4
Sex                1
Capital Gain       2
Capital Loss       2
Hours per week    40
Country            9
Name: 19146, dtype: int64


In [12]:
response = requests.post(url=service_link, json=make_signatured_json(pd.DataFrame(x).T))
prediction = np.array(response.json()["Prediction"])
print(f"Predicted label is {class_names[prediction[0][0]]}")

Predicted label is  <=50K


In [13]:
response = requests.post(url=anchor_link, json={"explained_instance": x.tolist(),
                                                "application_name" : "adult-salary-app",
                                                "config" : adult_anchor_config})

print(response.text)

{
  "coverage": 0.124, 
  "explanation": "Education == Dropout", 
  "precision": 0.963
}



In [14]:
print("Coverage of this explanation is ", response.json()['coverage'])
print("Precision of this explanation is ", response.json()['precision'])

response.json()['explanation'].split(" AND ")

Coverage of this explanation is  0.124
Precision of this explanation is  0.963


['Education == Dropout']

### Example 1

In [72]:
idx = np.random.choice(range(test_X.shape[0]))
x = test_X.iloc[idx]
print(x)

Age               41
Workclass          4
Education          5
Marital Status     0
Occupation         8
Relationship       0
Race               4
Sex                1
Capital Gain       2
Capital Loss       0
Hours per week    65
Country            9
Name: 405, dtype: int64


In [73]:
response = requests.post(url=service_link, json=make_signatured_json(pd.DataFrame(x).T))
prediction = np.array(response.json()["Prediction"])
print(f"Predicted label is {class_names[prediction[0][0]]}")

Predicted label is  >50K


In [74]:
response = requests.post(url=anchor_link, json={"explained_instance": x.tolist(),
                                                "application_name" : "adult-salary-app",
                                                "config" : adult_anchor_config})

In [75]:
print(response.text)

{
  "coverage": 0.009, 
  "explanation": "Occupation == White-Collar AND Capital Loss == High", 
  "precision": 0.975
}



In [76]:
print("Coverage of this explanation is ", response.json()['coverage'])
print("Precision of this explanation is ", response.json()['precision'])

Coverage of this explanation is  0.009
Precision of this explanation is  0.975


In [77]:
response.json()['explanation'].split(" AND ")

['Occupation == White-Collar', 'Capital Loss == High']

### Example 2

In [94]:
idx = np.random.choice(range(test_X.shape[0]))
x = test_X.iloc[idx]
print(x)

Age               56
Workclass          2
Education          6
Marital Status     0
Occupation         1
Relationship       0
Race               1
Sex                1
Capital Gain       2
Capital Loss       2
Hours per week    40
Country            7
Name: 22329, dtype: int64


In [95]:
response = requests.post(url=service_link, json=make_signatured_json(pd.DataFrame(x).T))
prediction = np.array(response.json()["Prediction"])
print(f"Predicted label is {class_names[prediction[0][0]]}")

Predicted label is  >50K


In [96]:
response = requests.post(url=anchor_link, json={"explained_instance": x.tolist(),
                                                "application_name" : "adult-salary-app",
                                                "config" : adult_anchor_config})
print(response.text)

{
  "coverage": 0.001, 
  "explanation": "Hours per week > 36.0 AND Capital Loss == None AND Sex ==  Male AND Education == Prof-School AND Workclass ==  Local-gov", 
  "precision": 1.0
}



In [118]:
x.tolist()

[45, 4, 4, 2, 6, 3, 4, 1, 2, 2, 45, 9]

### Example 3

In [97]:
idx = np.random.choice(range(test_X.shape[0]))
x = test_X.iloc[idx]
print(x)

Age               37
Workclass          4
Education          4
Marital Status     2
Occupation         2
Relationship       2
Race               2
Sex                1
Capital Gain       2
Capital Loss       2
Hours per week    40
Country            9
Name: 21521, dtype: int64


In [98]:
response = requests.post(url=service_link, json=make_signatured_json(pd.DataFrame(x).T))
prediction = np.array(response.json()["Prediction"])
print(f"Predicted label is {class_names[prediction[0][0]]}")

Predicted label is  <=50K


In [99]:
response = requests.post(url=anchor_link, json={"explained_instance": x.tolist(),
                                                "application_name" : "adult-salary-app",
                                                "config" : adult_anchor_config})
print(response.text)

{
  "coverage": 0.03, 
  "explanation": "Relationship ==  Other-relative", 
  "precision": 0.993
}

